In [5]:
# Top level run file
# Run the code from here

# Import the user defined functions
from functools import partial

from func_mors_dfs import create_mors_df_dict, filter_bab
from func_read_data import read_data
from func_score_agg import score_agg
from helper_functions import parse_mors_datestring, reduce_df_dict, parse_binary_to_boolean
from subfunctions.sub_func_remove_cols_by_pattern import remove_cols_by_pattern
from subfunctions.sub_func_transform_column_values import transform_column_values

# File path and names. Names are used as keys in the data_dict
path = '/Users/ellengoddard/Desktop/working_data/'
f_names = ['core_10_table', 'mors_table', 'pai_table_v2',
           'swemwebs_table', 'whooley_table']
ext = '.csv'
# List of NaN indicators used on read in of data
na_list = [999, '', 'Not applicable', 'NULL', 'Undefined', 'Unknown']

# Read in all datasets
data_dict = read_data(path, f_names, ext, na_list)

# Run the sub functions to aggregate the scores for each measure
# and invert where necessary so a higher score is always better
data_dict = score_agg(data_dict, f_names)

# Define MORS dataset 
mors_df = data_dict['mors_table']

# Remove Baby Steps data from MORS 
mors_df_filtered = filter_bab(mors_df)

# Separate MORS into dataframes based on assessment tool 
mors_df_dict = create_mors_df_dict(mors_df_filtered)

# Combine the original data_dict with the mors_df_dict
combined_data_dict = {**data_dict, **mors_df_dict}

# Remove columns with column names of pattern "question_"
combined_dict_no_questions = reduce_df_dict(combined_data_dict, partial(remove_cols_by_pattern, pattern="question_"))

# Parse all datestrings into datetime64
column_names = ["assessment_date", "ptt_date_reached_dosage", "cos_date_reached_dosage", "bab_date_reached_dosage"]
partial_transform_column_vals = partial(transform_column_values, column_names=column_names, function=parse_mors_datestring)
combined_dict_parsed_dates = reduce_df_dict(combined_dict_no_questions, partial_transform_column_vals)

# Parse 0 and 1s into booleans
column_names = ["bab_reached_dosage_yn", "ptt_reached_dosage_yn", "cos_reached_dosage_yn", "bab_reached_dosage_yn"]
partial_transform_column_vals_bool_parse = partial(transform_column_values, column_names=column_names, function=parse_binary_to_boolean)
combined_dict_parsed_bools = reduce_df_dict(combined_dict_parsed_dates, partial_transform_column_vals_bool_parse)

# combined_dict_parsed_bools["COS - MORSC"]
combined_dict_parsed_bools["pai_table_v2"]

Failed to apply function parse_mors_datestring to a value in column assessment_date in DataFrame core_10_table - does a row have a blank cell?
Failed to apply function parse_mors_datestring to a value in column ptt_date_reached_dosage in DataFrame core_10_table. Error: 'ptt_date_reached_dosage' - is the column missing?
Failed to apply function parse_mors_datestring to a value in column cos_date_reached_dosage in DataFrame core_10_table. Error: 'cos_date_reached_dosage' - is the column missing?
Failed to apply function parse_mors_datestring to a value in column bab_date_reached_dosage in DataFrame core_10_table. Error: 'bab_date_reached_dosage' - is the column missing?
Failed to apply function parse_mors_datestring to a value in column assessment_date in DataFrame mors_table - does a row have a blank cell?
Failed to apply function parse_mors_datestring to a value in column ptt_date_reached_dosage in DataFrame mors_table - does a row have a blank cell?
Failed to apply function parse_mors

---

Applying remove_cols_by_pattern to DataFrame 'core_10_table':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   leap_user_key        320 non-null    int64  
 1   assessment_date      320 non-null    object 
 2   assessment_tool      320 non-null    object 
 3   event                320 non-null    object 
 4   question_1           315 non-null    float64
 5   age_at_registration  316 non-null    float64
 6   ethnicity_b          237 non-null    object 
 7   home_language        283 non-null    object 
 8   lone_parent          12 non-null     object 
 9   lsoacode             312 non-null    object 
 10  ward_name            312 non-null    object 
 11  ward_code            312 non-null    object 
 12  borough              312 non-null    object 
 13  lsoaname             312 non-null    object 
 14  imdscore             31

,leap_user_key,assessment_date,assessment_tool,event,age_at_registration,ethnicity_b,home_language,lone_parent,lsoacode,ward_name,...,decile_1,imd_local_dec,imd_local_quint,mergestatus,bab_total_dosage,bab_reached_dosage_yn,bab_date_reached_dosage,user_has_pre_and_post,total,total_final
0,365389,2022-01-31,BAB - PAI,BAB - Home Visit 1,21.0,Black\r,English,Yes,E01003121,Stockwell,...,8.0,3.0,2.0,Matched (3),2.0,False,NaT,0,35.0,35.0
1,365623,2022-10-31,BAB - PAI,BAB - Antenatal Session 6,33.0,White\r,English,No,E01003186,Vassall,...,3.0,8.0,4.0,Matched (3),8.0,True,2022-10-26,1,52.0,52.0
2,365623,2022-08-24,BAB - PAI,BAB - Home Visit 1,33.0,White\r,English,No,E01003186,Vassall,...,3.0,8.0,4.0,Matched (3),8.0,True,2022-10-26,1,48.0,48.0
3,366132,2022-02-25,BAB - PAI,BAB - Antenatal Session 6,39.0,White\r,Other,NaN,E01003045,Coldharbour,...,10.0,1.0,1.0,Matched (3),7.0,True,2022-02-08,1,58.0,58.0
4,366132,2021-12-15,BAB - PAI,BAB - Home Visit 1,39.0,White\r,Other,NaN,E01003045,Coldharbour,...,10.0,1.0,1.0,Matched (3),7.0,True,2022-02-08,1,51.0,51.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,509466,2023-11-15,BAB - PAI,BAB - Antenatal Session 6,29.0,Black\r,English,No,E01003182,Vassall,...,6.0,5.0,3.0,Matched (3),6.0,True,2023-11-15,0,52.0,52.0
337,509479,2023-11-15,BAB - PAI,BAB - Antenatal Session 6,36.0,Asian\r,Chinese,No,E01003183,Vassall,...,7.0,4.0,2.0,Matched (3),6.0,True,2023-11-15,0,72.0,72.0
338,509481,2023-11-14,BAB - PAI,BAB - Antenatal Session 6,36.0,White\r,Portuguese,No,E01003126,Stockwell,...,7.0,4.0,2.0,Matched (3),6.0,True,2023-11-14,0,48.0,48.0
339,509485,2023-11-15,BAB - PAI,BAB - Antenatal Session 6,38.0,Black\r,English,No,E01003171,Tulse Hill,...,6.0,5.0,3.0,Matched (3),5.0,False,NaT,0,69.0,69.0
